In [1]:
import requests
import time
import os
import pandas as pd
import numpy as np
import random
import re
import regex

# !pip3 install -U spacy
# import spacy

from time import sleep
from os import path
from pandas import DataFrame
from bs4 import BeautifulSoup
from random import randint

In [2]:
def get_contents(soup, content_text):
  try:
    parents_blacklist=['[document]','html','head',
                       'style','script','body',
                       'section','tr',
                       'td','label','ul','header',
                       'aside']
    content=''
    text=soup.find_all(text=True)
    
#     print(text)
    
    for t in text:
        if t.parent.name not in parents_blacklist and len(t) > 4:
            content=content+t+' '
    content_text.append(content)
  except Exception:
    content_text.append('')
    pass


In [3]:
def localized(text):
#     print(text)
    contacts = r'\(?[0-9]{3}\)?\/?\.?-?\s?[0-9]{3}-?\.?-?\s?[0-9]{4}\b'
#     address = r'(?<!\S)\b([1-9][0-9]{,4}\s*-\s*)?[1-9][0-9]{,4} (?:[A-Z][a-zA-Z]*,?\s*){,5} Chicago,?[\w\s,]{,3}\s*[0-9]{,6}?\b'
    address = r'(?<!\S)\b([1-9][0-9]{,4}\s*-\s*)?[1-9][0-9]{,4} (?:[a-zA-Z0-9&]*,?\s*){,5} (Chicago|CHICAGO),?[\w\s,]{,3}\s*[0-9]{,6}?\b'
    address_2 = "\d+[ ](?:[A-Za-z0-9.-]+[ ]?)+(?:Avenue|Lane|Road|Boulevard|Drive|Street|Ave|Dr|Rd|Blvd|Ln|St)\.?,?\s*(Chicago|CHICAGO),?[\w\s,]{,3}\s*[0-9]{,6}?\b?"
#     bed_range = r'(?<!\S)\bStudio\s*-\s*[1-9]\s*[a-zA-Z]+\b|(?<!\S)\b[1-9]\s*-\s*[1-9]\s*[a-zA-Z]+\b|\(?<!\S)\bStudio$|\s*[1-9]\s*[a-zA-Z]+\b|(?<!\S)\bNo Availability\b'
#     bed_range = r'(?<!\S)\bStudio\s*-\s*[1-9]\s*[a-zA-Z]+\b|(?<!\S)\b[1-9]\s*-\s*[1-9]\s*[a-zA-Z]+\b|(?<!\S)\bStudio\b|(?<!\S)\b[1-9]\s*[a-zA-Z]+\b|(?<!\S)\bNo Availability\b'
    bed_range = r'((?<!\S)\b(Studio|[1-9])\s*-\s*[1-9]\s*(Bed|Br|Beds|Bedrooms)\b)'
    prices = r'\$[1-9]?[0-9]{,3},?[0-9]{,3}\s*-\s*\$?[1-9]?[0-9]{,3},?[0-9]{,3}|\$[1-9]?[0-9]{,3},?[0-9]{,3}$|Call for Rent\b'
    address = address_2
    output_final = []
    for i in re.finditer(address, text):
        property_info = {'Address': None, 'Price': None, 'Beds': None, 'Contact' : None}
        count = 0
        indexlocation= i.span()
        startindex= i.start()
        endindex= i.end()
        if indexlocation[0] >= 0 and indexlocation[1] + 100 < len(text):
            out = text[indexlocation[0]:indexlocation[1]+250] 
#             print("\nBLOCK", out)
            if re.search(address, out):
                property_info['Address'] = re.search(address, out).group()
                count += 1
            if re.search(prices, out):
                property_info['Price'] = re.search(prices, out).group()
                count += 1
            if re.search(bed_range,out):
                property_info['Beds'] = re.search(bed_range, out).group()
                count += 1
            if re.search(contacts,out):
                property_info['Contact'] = re.search(contacts, out).group()
                count += 1
            if count > 2:
                output_final.append(property_info)
#             time.sleep(5)
    merged = pd.DataFrame(output_final)
    print(merged)
    return merged

In [ ]:
links_df = pd.read_csv("results/links.csv")
# print(links_df)
for i, row in links_df.iterrows():
    webpage = row['Links']
    stamp = webpage[28:42]
    print(webpage, stamp)
    page=requests.get(webpage)
    soup=BeautifulSoup(page.text,'html.parser')
    content_text = []
    get_contents(soup, content_text)
    output_df = localized(content_text[0])
    file_name = "results_3/" + stamp + ".csv"
    output_df.to_csv(file_name)
    print("Saved to ", file_name)